## Informe Trabajo Práctico 3

#### Agustina Colazo

####  INTRODUCCIÓN

El objetivo de este informe es acceder a grandes conjuntos de datos 
observacionales y analizarlos estadisticamente, utilizando una Base de Datos y a traves del lenguaje: Structured Query Language (SQL). Una Base de datos es un lugar donde se almacenan sistematicamente, grandes conjuntos de datos pertenecientes a un mismo contexto. A su vez, se puede acceder a un subconjunto de datos con ciertas caracteristicas en comun, asi se obtiene una Tabla. La mismas esta compuestas por Campos y Registros. Los Campos son la minima unidad de informacion, se trata de una caracteristica en particular por ejemplo, Masa, Distancia, Tipo Espectral, etc y se corresponde a cada columna de la Tabla. Por otro lado, cada objeto con sus caracteristicas forma un Registro, es decir que es un onjeto unico de datos implicitamente estructurados en una Tabla y se corresponde a una fila de la misma.

A continuacion, se utiliza el catalogo de exoplanetas de la base de datos exoplanets.eu y se analizan algunas de las principales caracteristicas de dichos sistemas tales como su masa, radio, periodo orbital, etc. En particular, se estudia la distribucion de masas y periodos orbitales para planetas descubiertos con diferentes tecnicas observacionales. Se analiza si la distribucion de distancias a estrellas con exoplanetas es una distribucion Gaussiana o no. Por ultimo, se propone un modelo para la relacion entre las masas y radios de los exoplanetas y se realiza un ajuste del modelo.

Se instala el paquete pyvo necesario para acceder a la base de datos (de manera remota) y hacer el query para obtener una tabla con los datos de interes. Se realiza un query de prueba.

In [6]:
pip install pyvo

/usr/bin/python3: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install scikit-learn

/usr/bin/python3: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# se descargar las librerias necesarias

import pyvo as vo
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp
import pandas as pd
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pyvo'

In [ ]:
# Se desea obtener una tabla/lista de exoplanetas con las siguientes propiedades: Masa, Periodo Orbital, Tipo de Estrella del Sistema

# Utilizo pyvo y selecciono el servidor a partir del cual se obtendran los datos
service = vo.dal.TAPService("http://voparis-tap-planeto.obspm.fr/tap")

# defino el query, es decir los datos de interes que se van a buscar en la base de datos
# Aca se utiliza SQL (Structured Query Language)   
query = "SELECT target_name,mass,period,star_spec_type FROM exoplanet.epn_core"

results = service.search(query)  # obtengo una tabla con los datos del query
print(type(results))  # este es una tabla tipo pyvo.dal.tap.TAPResults  (un posible formato de tabla)
table = results.to_table()  # ahora la transformo en una 'astropy table'  (otro formato)
#datas = table.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
vot = results.votable
vot.to_xml("Prueba_Exoplanets_Data.xml") # esto me baja una tabla con los datos a la compu
print(table)

#### Ejercicio 6: Distribución de Masas y Períodos Orbitales

Se examina las distribucions de masas y periodos orbitales de exoplanetas, detectados con las siguientes tecnicas observacionles: Imaging, Radial Velocity, Microlensing, Primary Transit, Astrometry, Transit Timing Variation (TTV) y Pulsar. Se intenta determinar si la tecnica utilizada introduce un sesgo las caracteristicas de los exoplanetas detectados.


In [ ]:
query0 = "SELECT target_name,detection_type FROM exoplanet.epn_core WHERE mass<20"
results0 = service.search(query0)  # obtengo una tabla con los exoplanetas y su metodo de deteccion
table0 = results0.to_table()  # ahora la transformo en una 'astropy table'  (otro formato)
data = table0.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data)
#print(data.shape)


# Ahora se analizaran las distribuciones de las masas

# planetas detectados por Imaging
query1 = "SELECT target_name,detection_type,mass,period FROM exoplanet.epn_core WHERE detection_type='Imaging' and mass<20" 
#print(type(query))  me dice que es un string
planets_imaging = service.search(query1)  # obtengo una tabla con los datos del query
table1 = planets_imaging.to_table() # ahora la transformo en una astropy table que tiene el metodo to_pandas para pasar a un dataframe
data1 = table1.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
m_img = data1['mass']
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(m_img, density=True, histtype = 'step', lw = 3, label = 'Imaging')

# planetas detectados por Radial Velocity
query2 = "SELECT target_name,detection_type,mass,period FROM exoplanet.epn_core WHERE detection_type='Radial Velocity' and mass<20" 
planets_radvel = service.search(query2)  # obtengo una tabla con los datos del query
table2 = planets_radvel.to_table()  # ahora la transformo en una astropy table que tiene el metodo to_pandas para pasar a un dataframe
data2 = table2.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
m_rv = data2['mass']
#print(m_radvel)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(m_rv, density=True, histtype = 'step', lw = 3 ,label = 'Radial Velocity')

# planetas detectados por Timing  (NO HAY NINGUNO)
query3 = "SELECT target_name,detection_type,mass,period FROM exoplanet.epn_core WHERE detection_type='Timing' and mass<20" 
planets_timing = service.search(query3)  # obtengo una tabla con los datos del query
table3 = planets_timing.to_table()  # ahora la transformo en una 'astropy table'  (otro formato)
data3 = table3.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
print(data3)
print('No hay planetas detectados por Timing')


# planetas detectados por Microlensing
query4 = "SELECT target_name,detection_type,mass,period FROM exoplanet.epn_core WHERE detection_type='Microlensing' and mass<20" 
#print(type(query))  me dice que es un string
planets_micro = service.search(query4)  # obtengo una tabla con los datos del query
table4 = planets_micro.to_table()  # ahora la transformo en una 'astropy table'  (otro formato)
data4 = table4.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data4)
m_micro = data4['mass']
#print(m_micro)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(m_micro, density = True, histtype = 'step', lw = 3, label = 'Microlensing')

# Grafico todo junto
plt.xlabel('Masa de Exoplanetas [M_Jup]')
plt.ylabel('Frecuencias')
plt.legend()
plt.title('Distribucion de Masa de los exoplanetas detectados con distintas tecnicas')
plt.show()

# planetas detectados por Primary Transit
query5 = "SELECT target_name,detection_type,mass,period FROM exoplanet.epn_core WHERE detection_type='Primary Transit' and mass<20" 
#print(type(query))  me dice que es un string
planets_transit = service.search(query5)  # obtengo una tabla con los datos del query
table5 = planets_transit.to_table()  # ahora la transformo en una 'astropy table'  (otro formato)
data5 = table5.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data5)
m_transit = data5['mass']
#print(m_transit)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(m_transit, density = True, histtype = 'step', lw = 3 , color='red',label = 'Primary Transit')


# planetas detectados por Astrometry
query6 = "SELECT target_name,detection_type,mass,period FROM exoplanet.epn_core WHERE detection_type='Astrometry' and mass<20" 
#print(type(query))  me dice que es un string
planets_astrom = service.search(query6)  # obtengo una tabla con los datos del query
table6 = planets_astrom.to_table()  # ahora la transformo en una 'astropy table'  (otro formato)
data6 = table6.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data6)
m_astrom  = data6['mass']
#print(m_astrom)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(m_astrom, density = True, histtype = 'step' , lw = 3, color='darkblue',label = 'Astrometry')

# planetas detectados por TTV
query7 = "SELECT target_name,detection_type,mass,period FROM exoplanet.epn_core WHERE detection_type='TTV' and mass<20" 
#print(type(query))  me dice que es un string
planets_TTV = service.search(query7)  # obtengo una tabla con los datos del query
table7 = planets_TTV.to_table()  # ahora la transformo en una 'astropy table'  (otro formato)
data7 = table7.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data7)
m_TTV  = data7['mass']
#print(m_TTV)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(m_TTV, density = True, histtype = 'step',lw = 3 ,color='black' ,label = 'TTV')

# planetas detectados por Pulsar
query8 = "SELECT target_name,detection_type,mass,period FROM exoplanet.epn_core WHERE detection_type='Pulsar' and mass<20" 
#print(type(query))  me dice que es un string
planets_pulsar = service.search(query8)  # obtengo una tabla con los datos del query
table8 = planets_pulsar.to_table()  # ahora la transformo en una 'astropy table'  (otro formato)
data8 = table8.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data8)
m_pul = data8['mass']
#print(m_pul)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(m_pul, density = True, histtype = 'step', lw = 3,color='deeppink', label = 'Pulsar')

# Grafico todo junto
plt.xlabel('Masa de Exoplanetas [M_Jup]')
plt.ylabel('Frecuencias')
plt.legend()
plt.title('Distribucion de Masa de los exoplanetas detectados con distintas tecnicas')
plt.show()

Se puede observar que la mayoria de las tecnicas observcionales detectan muchos exoplanetas con masas en el rango $ 0 < M < 2.5 M_{Jup}$ ya que la mayoria de las distribuciones tienen un pico alli. Sin embargo, en el caso de la tecnica observacional 'Imaging' (deteccion del exoplaneta en imagenes astronomicas) esto no se cumple, y se nota que el pico de dicha distribucion se da alrededor de $ 11\,M_{Jup} $ Con lo cual se puede advertir un sesgo en esta tecnica, ya que tiende a detectar mayoritariamente exoplanetas de masas intermedias, i.e en un rango de $7.5 < M < 15 M_{Jup}$.
Por otro lado, si bien la mayoria de las distribuciones tienen un pico en el primer bin, (i.e muchos exoplanetas de baja masa detectados)  es notable que la tecnica TTV tiene un pico mucho mas grande y practicamente no se han detectado exoplanetas de masas mayores a $ 7 M_{Jup}$ con ese metodo. Con lo cual es probable que exista un sesgo en dicha tecnica, de tal forma que tiende a detectar solamente planetas de baja masa (i.e principalmente masas menores a $ 1 M_{Jup}$).


In [ ]:
## Ahora se estudian las distribuciones de los periodos oribitales

# planetas detectados por Imaging 
p_img = data1['period']
#print(p_img)
#aca simplemente hago el histograma de los datos con sus frecuencias , Escala logaritmica en los periodos
p1 = np.log(p_img)
plt.hist(p1, density=True, histtype = 'step', lw = 3, label = 'Imaging')


# planetas detectados por Microlensing
p_micro = data4['period']
#print(p_micro)
p4 = np.log(p_micro)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(p4, density = True, histtype = 'step', lw = 3, label = 'Microlensing')

# planetas detectados por Primary Transit
p_transit = data5['period']
#print(p_transit)
p5 = np.log(p_transit)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(p5, density = True, histtype = 'step', lw = 3, label = 'Primary Transit')

# planetas detectados por Astrometry
p_astrom  = data6['period']
#print(p_astrom)
p6 = np.log(p_astrom)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(p6, density = True, histtype = 'step', lw = 3, label = 'Astrometry')

plt.xlabel('Logaritmo del Periodo Orbital [log(dias)]')
plt.ylabel('Frecuencias')
plt.legend()
plt.title('Distribucion de Periodos de los exoplanetas detectados con distintas tecnicas en escala logaritmica')
plt.show()

# planetas detectados por TTV
p_TTV  = data7['period']
#print(p_TTV)
p7 = np.log(p_TTV )
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(p7, density = True, histtype = 'step', lw = 3, color='darkblue' , label = 'TTV')


# planetas detectados por Pulsar
p_pul  = data8['period']
#print(p_pul)
p8 = np.log(p_pul)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(p8, density = True, histtype = 'step', lw = 3, color='black',label = 'Pulsar')

# planetas detectados por Radial Velocity
p_radvel = data2['period']
#print(p_radvel)
p2 = np.log(p_radvel)
#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(p2, density=True, histtype = 'step', lw = 3, color='deeppink', label = 'Radial Velocity')

plt.xlabel('Logaritmo del Periodo Orbital [log(dias)]')
plt.ylabel('Frecuencias')
plt.legend()
plt.title('Distribucion de Periodos de los exoplanetas detectados con distintas tecnicas en escala logaritmica')
plt.show()

En este caso, no se observa un patron general en las distribuciones. Con algunas de las tecnicas, se han detectado exoplanetas abarcando un amplio rango de distintos periodos orbitales. Por ejemplo con el metodo Imaging los posibles periodos abarcan el rango $5 < \log(P[dias]) < 15$, con el de Radial Velocity y Pulsar se abarca un rango de $-3 < \log(P[dias]) < 10$. 
En cambio, con otras tecnicas se detectan exoplanetas con ciertos periodos orbitales que abarcan un intervalo mas acotado y podrian estar sesgadas. Tal es el caso de los exoplanetas detectados por el metodo Microlensing (donde $7 < \log(P[dias]) < 8.5$) o Astrometry (donde $5 < \log(P[dias]) < 9$).

Ademas, existe una gran diversidad en cuanto al periodo mas probable de cada distribucion. Se observa que algunas no tienen un pico bien definido en la distribucion de los periodos orbitales; tal es el caso de la tecnica 'Astrometry', 'Pulsar' y 'Radial Velocity'. En estos casos no es claro que haya una tendencia a detectar planetas con cierto periodo orbital. En cambio, con la tecnica 'Primary Transit' se observa una tendencia a detectar exoplanetas con periodos alrededor del valor $\log(P[dias])= 1.5 $ (i.e exoplanetas con periodos mas bien cortos entre 1 y 12 dias), mientras que con la tecnica 'Imaging' la mayoria de los exoplanetas tienen periodos alrededor de  $\log(P[dias])= 11 $ (i.e exoplanetas con periodos mas largos, entre 22.000 y 160.000 dias). Por ultimo, con la tecnica 'Microlensing' se observa que la mayoria de los exoplanetas tienen un periodo orbital del orden de $\log(P[dias])= 8 $ (i.e exoplanetas con periodos, entre 1000 y 5000 dias). 

#### Ejercicio 7: Distribución de Distancias a Estrellas con Exoplanetas

A continuacion, se busca determinar si la distribucion de semiejes mayores de las orbitas de los exoplanetas es consistente con una distribucion (teorica) Gaussiana. Para esto se utilizara el Metodo Kolmogorv-Smirnov para una muestra, (one-sample KS test). Este metodo consiste en estabalecer una hipotesis nula y alternativa, en este caso serian:

Hipotesis Nula: $\ H_0=$  La muestra empirica sigue una distribucion Gaussiana con un nivel de significancia $\alpha=0.05$

Hipotesis Alternativa: $\ H_a=$  La muestra no procede de una distribucion Gaussiana con un nivel de significancia $\alpha=0.05$

Luego se calcula un estadistico definido como : 
$$D=max |F_{1}(x)-F_{2}(x)|\\ x\in I$$

$0 ≤ D ≤ 1$; donde $I$ es algun intervalo y $F_{1}(x)$ y $F_{2}(x)$ son las funciones acumuladas de la distribucion empirica y  Gaussiana respectivamente. Es decir que se calcula la distancia maxima que existe entre ambas funciones para algun $x_i$ y se compara ese valor con otro valor de referencia, el cual depende de los grados de libertad $n$ y el nivel de significancia $\alpha$. Si el estadistico calculado supera el valor de referencia, entonces es poco probable (para el nivel de significancia dado) que la muestra siga la distribucion teorica establecida en la hipotesis nula. 

$$ Zona\,\,de\,\,aceptacion: D < D_{KS,\alpha;n}\\ $$ 
$$ Zona\,\,de\,\,rechazo: D \geq D_{KS,\alpha;n}\\ $$

Ademas, se puede utilizar el valor-p o p-value que provee este metodo:

$$ Zona\,\,de\,\,aceptacion: p-value > \alpha \\ $$ 
$$ Zona\,\,de\,\,rechazo: p-value \leq \alpha \\ $$

In [ ]:
# Obtengo los semiejes mayores (UA)
query9 = "SELECT target_name,semi_major_axis FROM exoplanet.epn_core WHERE mass<20" 
#print(type(query))  me dice que es un string
major_axis = service.search(query9)  # obtengo una tabla con los datos del query
table9 = major_axis.to_table()  #esto es una astropy table que se puede transformar en dataframe
data9 = table9.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data9)
# Inner Join
exo_axis  = pd.merge(left = data9, right = data, how = 'inner',on = 'target_name')
#print('este es el tamño de la tabla act',exo_axis.shape)
#print(exo_axis)

dist = data9['semi_major_axis'][~data9['semi_major_axis'].isna()] # saco los Nan
#print(dist)
a = dist.loc[dist < 2]
#print(a)

#aca simplemente hago el histograma de los datos con sus frecuencias 
plt.hist(a, bins='auto',density = True)
plt.xlabel('Semieje Mayor [UA]')
plt.ylabel('Frecuencias')
plt.title('Distribucion de Semieje Mayor de las Orbitas de Exoplanetas')

sp.kstest(a,'norm')


Se obtiene como resultado $D = 0.5017553403698662 $ y $p-value = pvalue=8.900347328192646e-270$, dado que $p-value < \alpha$, se rechaza la hipotesis nula, es decir que no se tiene suficiente evidencia para concluir que las distancias a las estrellas con exoplanetas sigue una distribucion Gaussiana.

#### Ejercicio 8: Relacion Masa versus Radio de Exoplanetas

EL objetivo de este ejercicio es buscar una relacion entre la masa y el radio de exoplanetas, para luego proponer y ajustar un modelo que reproduzca dicha relacion.

Para esto se se grafican los pares $ (\,\log(M) ; \log(R)\,) $ y se propone una ley de potencias para la relacion entre la masa y el radio:

$$ M(R) = \beta R^\alpha $$   Con lo cual se tiene que:

$$ log(M) = \alpha \log(R) + \log(\beta) $$ 
Es decir que basta realizar un ajuste lineal de la forma $$ y = A x + Β $$ 
donde $ A = \alpha\,\,$ y $\,\,Β = \log(\beta) $.

Por lo tanto, se ajusta una recta a los pares de puntos graficados para obtener los parametros del modelo $\alpha$ y $\beta$. 

In [ ]:
# Obtengo las masas y radios de los exoplanetas
query10 = "SELECT target_name,mass,radius FROM exoplanet.epn_core WHERE mass<20"
results_10 = service.search(query10)  # obtengo una tabla con los datos del query
table10 = results_10.to_table()  # ahora la transformo en una 'astropy table' 
data10 = table10.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data10) 
# Inner Join
exo_mr  = pd.merge(left = data10, right = exo_axis, how = 'inner',on = 'target_name' )
#print(exo_mr)
exo_mr.to_csv('Exoplanets_Data.csv')   #para guardar datos 

query11 = "SELECT target_name,mass,radius FROM exoplanet.epn_core WHERE mass<20 and mass!='Nan' and radius!='Nan'"
results_11 = service.search(query11)  # obtengo una tabla con los datos del query
table11 = results_11.to_table()  # ahora la transformo en una 'astropy table' 
data11 = table11.to_pandas()  # ahora la transformo en un 'data frame' usando pandas (otro formato)
#print(data11) 

 
#aca simplemente hago el grafico de los datos todo en escala logaritmica
m = np.log(data11['mass'])
r = np.log(data11['radius'])
#print(m)
#print(r)

plt.plot(r , m , '.' , color='r', markersize=2)
plt.xlabel('Log(Radio)  [log(R_Jup)]')
plt.ylabel('Log(Masa)  [log(M_ Jup)]')
plt.title('Relacion log(Masa) vs log(Radio) de Exoplanetas')

#Hago un ajuste lineal
ajuste = np.polyfit(r,m,1)
print(ajuste) # estos son los parametros del ajuste
t = np.linspace(-4,2)
#print(t)    
plt.plot(t, ajuste[0]*t+ajuste[1], 'b') # esta es la recta que predice el modelo
plt.grid()

alpha=ajuste[0]
beta = np.exp(ajuste[1])
print('Los valores de los parametros alpha y beta son respectivamente:',alpha,beta)


A partir del ajuste se obtuvieron los valores de $\alpha=1.91$ y $\beta=0.75$; por lo tanto el modelo predice una relacion entre la masa y y el radio de exoplanetas de la forma: 

$$ M(R) = 0.75R^{1.91} $$

A continuacion, se toma el 80% de los datos (radio, masa) de exoplanetas y se le ajustan polinomios de grado 0 a 10, luego con los pararmetros de los ajustes se obtienen 2 polinomios, uno evaluado en el 80% de la poblacion (train) con el cual se realizo el ajuste, y otro evaluado en el 20% restante (datos que no se usaron para hacer el ajuste ). A continuacion se calcula y grafica el error para ambas muestras y para cada grado de polinomio utilizado. En aquel punto donde las curvas se separan notablemente entre si significa que ese es el grado maximo de polinomio que se deberia utilizar; ya que si se utiliza un polinomio de grado mayor para realizar el ajuste se estaria sobreajustando los datos. 

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(r,m,test_size=0.2)  

# el 80 % de los datos van a estar en (x_train,y_train) y el 20% restante de
#los datos en (x_test,y_test)
print(x_train.shape)
print(x_test.shape)

e_train =[]
e_test= []
for i in range(0,11):
    p = np.polyfit(x_train,y_train,i)  #esto contiene  los parametros del polinomio
    poli = np.poly1d(p) # aca le doy los parametros y me devuelve ya el polinomio con los pararmetros
    poli(x_train)
    poli(x_test)
    e_train.append(  (np.sum((poli(x_train)-y_train)**2))/len(x_train) )
    e_test.append( (np.sum((poli(x_test)-y_test)**2))/len(x_test) )


plt.plot(e_train, label = 'e_train')
plt.plot(e_test, label = 'e_test')
plt.legend()  
plt.xlabel('Grado de Polinomio')
plt.ylabel('Error')
plt.semilogy()
plt.show()


Se observa que efectivamente las curvas se separan para n=1 (n es el grado de polinomio). Por lo tanto, fue correcto realizar un ajuste lineal a los datos de las masas vs radio de exoplanetas.

####  CONCLUSIONES

En este trabajo se analizaron distintas propiedades de los exoplanetas descubiertos, utilizando la Base de Datos exoplanet.eu.
En primer lugar, existen ciertos sesgos introducidos por algunas de las tecnicas de deteccion de exoplanetas en el tipo de propiedades que mas se observan en los mismos. Se encontro que efectivamente algunos metodos observacionales tienden a detectar planetas mas masivos (Imaging) o bien menos masivos (TTV) en compararcion con las demas tecnicas. Ademas, con la tecnica 'Primary Transit' se observa una tendencia a detectar exoplanetas con periodos mas bien cortos (entre 1 y 12 dias), mientras que con la tecnica 'Imaging' se sueleen detetctar mas bien exoplanetas con periodos mas largos (entre 22.000 y 160.000 dias) en comparacion con las otras tecnicas.
Por otro lado, se grafico la distribucion de probabilidad para el semieje mayor de la orbita de los exoplanetas descubiertos y se la comparon con una distribucion gaussiana utilizando el test de Kolmogorov-Smirnov. Se encontro que no existe suficiente evidencia para concluir que la distribucion empirica obtenida sea consistente con una distribucion normal. Esto puede ser debido a que el semieje mayor efectivamente siga otra distribucion, o bien prodria ser por algun sesgo que introducen todas las tencicas de deteccion que hace que haya una tendencia a descubrir exoplanetas con semiejes entre 0 y 0.25 UA.
Por ultimo, se estudio la relacion entre la masa y el radio de los exoplanetas y ajustando un modelo en base a los datos se encontro que siguen la siguiente relacion $ M(R) = 0.75R^{1.91}$.


